import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve,
    roc_auc_score
)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

print("Libraries imported successfully!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. Setup and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve,
    roc_auc_score
)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

print("Libraries imported successfully!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

In [ ]:
# Load model comparison results
model_comparison = pd.read_csv('../models/model_metadata/model_comparison.csv')

print("Model Performance Comparison:")
print("="*100)
print(model_comparison[['Model', 'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1', 'Test_ROC_AUC']].to_string(index=False))

# Load all predictions
predictions = pd.read_csv('../models/model_metadata/all_predictions.csv')
print(f"\nPredictions loaded: {predictions.shape}")

In [ ]:
# Load test data for analysis
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').squeeze()

print(f"Test data loaded: {X_test.shape}")

## 2. Detailed Performance Comparison

In [ ]:
# Identify best model by different criteria
print("Best Models by Metric:")
print("="*80)

metrics = ['Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1', 'Test_ROC_AUC']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']

for metric, name in zip(metrics, metric_names):
    best_model = model_comparison.loc[model_comparison[metric].idxmax(), 'Model']
    best_score = model_comparison[metric].max()
    print(f"{name:12s}: {best_model:20s} ({best_score:.4f})")

# Overall recommendation based on ROC-AUC
recommended_model = model_comparison.loc[model_comparison['Test_ROC_AUC'].idxmax(), 'Model']
print(f"\n{'='*80}")
print(f"Recommended Model (based on ROC-AUC): {recommended_model}")
print(f"{'='*80}")

In [ ]:
# Performance improvement analysis
baseline_auc = model_comparison[model_comparison['Model'].isin(['Logistic Regression', 'Decision Tree'])]['Test_ROC_AUC'].max()
best_auc = model_comparison['Test_ROC_AUC'].max()
improvement = ((best_auc - baseline_auc) / baseline_auc) * 100

print(f"\nPerformance Improvement:")
print(f"  Baseline (best simple model): {baseline_auc:.4f}")
print(f"  Best model: {best_auc:.4f}")
print(f"  Improvement: {improvement:.2f}%")

## 3. Error Analysis

In [ ]:
# ROI calculation (compare to no model scenario)
# Assume without model, we catch 0% of delays
total_delays = predictions['y_true'].sum()
cost_without_model = total_delays * COST_FALSE_NEGATIVE

print("ROI Analysis:")
print("="*80)
print(f"Total delays in test set: {total_delays}")
print(f"Cost without prediction model: ${cost_without_model:,.0f}")
print(f"\\nCost savings with {recommended_model}:")

model_cost = business_df[business_df['Model'] == recommended_model]['Total_Cost'].values[0]
savings = cost_without_model - model_cost
roi = (savings / cost_without_model) * 100


print(f"  Model cost: ${model_cost:,.0f}")
print(f"  Savings: ${savings:,.0f}")
print(f"  ROI: {roi:.1f}%")

In [ ]:
# Create error analysis dataframe
error_analysis = X_test.copy()
error_analysis['y_true'] = predictions['y_true'].values
error_analysis['y_pred'] = predictions[best_pred_col].values
error_analysis['y_proba'] = predictions[best_proba_col].values

# Classify errors
error_analysis['prediction_type'] = 'Correct'
error_analysis.loc[(error_analysis['y_true'] == 1) & (error_analysis['y_pred'] == 0), 'prediction_type'] = 'False Negative'
error_analysis.loc[(error_analysis['y_true'] == 0) & (error_analysis['y_pred'] == 1), 'prediction_type'] = 'False Positive'

print("\nPrediction Type Distribution:")
print(error_analysis['prediction_type'].value_counts())
print(f"\nError rate: {(error_analysis['prediction_type'] != 'Correct').mean():.2%}")

In [ ]:
# Detailed confusion matrix with labels
cm = confusion_matrix(predictions['y_true'], predictions[best_pred_col])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['On-Time', 'Late'],
            yticklabels=['On-Time', 'Late'])
plt.title(f'Confusion Matrix - {recommended_model}')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Add percentages
total = cm.sum()
for i in range(2):
    for j in range(2):
        plt.text(j+0.5, i+0.7, f'({cm[i,j]/total*100:.1f}%)', 
                ha='center', va='center', fontsize=10, color='gray')

plt.tight_layout()
plt.show()

# Print detailed metrics
print(f"\nClassification Report - {recommended_model}:")
print("="*80)
print(classification_report(predictions['y_true'], predictions[best_pred_col], 
                          target_names=['On-Time', 'Late']))

In [ ]:
# Analyze false negatives (missed delays)
false_negatives = error_analysis[error_analysis['prediction_type'] == 'False Negative']
false_positives = error_analysis[error_analysis['prediction_type'] == 'False Positive']

print(f"\nFalse Negative Analysis (Missed Delays):")
print(f"  Count: {len(false_negatives)}")
print(f"  Avg prediction probability: {false_negatives['y_proba'].mean():.3f}")
print(f"  Probability range: {false_negatives['y_proba'].min():.3f} - {false_negatives['y_proba'].max():.3f}")

print(f"\nFalse Positive Analysis (False Alarms):")
print(f"  Count: {len(false_positives)}")
print(f"  Avg prediction probability: {false_positives['y_proba'].mean():.3f}")
print(f"  Probability range: {false_positives['y_proba'].min():.3f} - {false_positives['y_proba'].max():.3f}")

## 4. Business Impact Analysis

In [ ]:
# Define business costs (from config)
COST_FALSE_NEGATIVE = 5000  # Cost of missing a delay (expedited shipping, penalties)
COST_FALSE_POSITIVE = 500   # Cost of false alarm (staff time, unnecessary intervention)

# Calculate business costs for each model
business_costs = []

for model_name in model_comparison['Model']:
    pred_col = model_pred_map[model_name]
    
    fn = ((predictions['y_true'] == 1) & (predictions[pred_col] == 0)).sum()
    fp = ((predictions['y_true'] == 0) & (predictions[pred_col] == 1)).sum()
    
    total_cost = (fn * COST_FALSE_NEGATIVE) + (fp * COST_FALSE_POSITIVE)
    
    business_costs.append({
        'Model': model_name,
        'False_Negatives': fn,
        'False_Positives': fp,
        'FN_Cost': fn * COST_FALSE_NEGATIVE,
        'FP_Cost': fp * COST_FALSE_POSITIVE,
        'Total_Cost': total_cost
    })

business_df = pd.DataFrame(business_costs).sort_values('Total_Cost')

print("Business Cost Analysis:")
print("="*100)
print(business_df.to_string(index=False))

# Best model by business cost
best_business_model = business_df.iloc[0]['Model']
best_business_cost = business_df.iloc[0]['Total_Cost']
print(f"\nBest Model by Business Cost: {best_business_model} (${best_business_cost:,.0f})")

In [ ]:
# Visualize business costs
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Total cost comparison
axes[0].barh(business_df['Model'], business_df['Total_Cost']/1000)
axes[0].set_xlabel('Total Cost ($1000s)')
axes[0].set_title('Total Business Cost by Model')
axes[0].grid(True, alpha=0.3)

# Cost breakdown
x = np.arange(len(business_df))
width = 0.35

axes[1].bar(x - width/2, business_df['FN_Cost']/1000, width, label='False Negative Cost')
axes[1].bar(x + width/2, business_df['FP_Cost']/1000, width, label='False Positive Cost')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Cost ($1000s)')
axes[1].set_title('Cost Breakdown by Error Type')
axes[1].set_xticks(x)
axes[1].set_xticklabels(business_df['Model'], rotation=45, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../models/model_metadata/business_cost_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ROI calculation (compare to no model scenario)
# Assume without model, we catch 0% of delays
total_delays = predictions['y_true'].sum()
cost_without_model = total_delays * COST_FALSE_NEGATIVE

print("ROI Analysis:")
print("="*80)
print(f"Total delays in test set: {total_delays}")
print(f"Cost without prediction model: ${cost_without_model:,.0f}")
print(f"\nCost savings with {recommended_model}:")

model_cost = business_df[business_df['Model'] == recommended_model]['Total_Cost'].values[0]
savings = cost_without_model - model_cost
roi = (savings / cost_without_model) * 100

print(f"  Model cost: ${model_cost:,.0f}")
print(f"  Savings: ${savings:,.0f}")
print(f"  ROI: {roi:.1f}%")

## 5. Threshold Optimization

In [ ]:
# Analyze different thresholds for the best model
thresholds_to_test = np.arange(0.1, 0.9, 0.05)
threshold_results = []

for threshold in thresholds_to_test:
    y_pred_thresh = (predictions[best_proba_col] >= threshold).astype(int)
    
    fn = ((predictions['y_true'] == 1) & (y_pred_thresh == 0)).sum()
    fp = ((predictions['y_true'] == 0) & (y_pred_thresh == 1)).sum()
    tp = ((predictions['y_true'] == 1) & (y_pred_thresh == 1)).sum()
    tn = ((predictions['y_true'] == 0) & (y_pred_thresh == 0)).sum()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    total_cost = (fn * COST_FALSE_NEGATIVE) + (fp * COST_FALSE_POSITIVE)
    
    threshold_results.append({
        'Threshold': threshold,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Total_Cost': total_cost,
        'FN': fn,
        'FP': fp
    })

threshold_df = pd.DataFrame(threshold_results)

# Find optimal threshold by minimum cost
optimal_threshold = threshold_df.loc[threshold_df['Total_Cost'].idxmin(), 'Threshold']
optimal_cost = threshold_df['Total_Cost'].min()

print(f"Threshold Optimization Results:")
print(f"  Optimal threshold: {optimal_threshold:.2f}")
print(f"  Optimal cost: ${optimal_cost:,.0f}")
print(f"  Default threshold (0.5) cost: ${threshold_df[threshold_df['Threshold']==0.5]['Total_Cost'].values[0]:,.0f}")
print(f"  Cost improvement: ${threshold_df[threshold_df['Threshold']==0.5]['Total_Cost'].values[0] - optimal_cost:,.0f}")

In [ ]:
# Visualize threshold impact
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Precision-Recall tradeoff
axes[0, 0].plot(threshold_df['Threshold'], threshold_df['Precision'], label='Precision', marker='o')
axes[0, 0].plot(threshold_df['Threshold'], threshold_df['Recall'], label='Recall', marker='s')
axes[0, 0].plot(threshold_df['Threshold'], threshold_df['F1'], label='F1 Score', marker='^')
axes[0, 0].axvline(x=optimal_threshold, color='r', linestyle='--', label=f'Optimal ({optimal_threshold:.2f})')
axes[0, 0].set_xlabel('Threshold')
axes[0, 0].set_ylabel('Score')
axes[0, 0].set_title('Precision-Recall-F1 vs Threshold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Total cost vs threshold
axes[0, 1].plot(threshold_df['Threshold'], threshold_df['Total_Cost']/1000, marker='o', color='red')
axes[0, 1].axvline(x=optimal_threshold, color='g', linestyle='--', label=f'Optimal ({optimal_threshold:.2f})')
axes[0, 1].set_xlabel('Threshold')
axes[0, 1].set_ylabel('Total Cost ($1000s)')
axes[0, 1].set_title('Business Cost vs Threshold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# False negatives vs false positives
axes[1, 0].plot(threshold_df['Threshold'], threshold_df['FN'], label='False Negatives', marker='o')
axes[1, 0].plot(threshold_df['Threshold'], threshold_df['FP'], label='False Positives', marker='s')
axes[1, 0].axvline(x=optimal_threshold, color='r', linestyle='--', label=f'Optimal ({optimal_threshold:.2f})')
axes[1, 0].set_xlabel('Threshold')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Error Counts vs Threshold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Precision-Recall curve
precision_curve, recall_curve, _ = precision_recall_curve(predictions['y_true'], predictions[best_proba_col])
axes[1, 1].plot(recall_curve, precision_curve, linewidth=2)
axes[1, 1].set_xlabel('Recall')
axes[1, 1].set_ylabel('Precision')
axes[1, 1].set_title('Precision-Recall Curve')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../models/model_metadata/threshold_optimization.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Final Model Selection & Recommendations

In [ ]:
print("="*100)
print("FINAL MODEL SELECTION & DEPLOYMENT RECOMMENDATIONS")
print("="*100)

print(f"\n1. RECOMMENDED MODEL: {recommended_model}")
print(f"   {'='*80}")

# Get metrics for recommended model
rec_metrics = model_comparison[model_comparison['Model'] == recommended_model].iloc[0]
rec_business = business_df[business_df['Model'] == recommended_model].iloc[0]

print(f"\n   Performance Metrics:")
print(f"     - Test Accuracy:  {rec_metrics['Test_Accuracy']:.4f}")
print(f"     - Test Precision: {rec_metrics['Test_Precision']:.4f}")
print(f"     - Test Recall:    {rec_metrics['Test_Recall']:.4f}")
print(f"     - Test F1 Score:  {rec_metrics['Test_F1']:.4f}")
print(f"     - Test ROC-AUC:   {rec_metrics['Test_ROC_AUC']:.4f}")

print(f"\n   Business Metrics:")
print(f"     - False Negatives: {rec_business['False_Negatives']}")
print(f"     - False Positives: {rec_business['False_Positives']}")
print(f"     - Total Cost: ${rec_business['Total_Cost']:,.0f}")
print(f"     - Cost Savings vs No Model: ${savings:,.0f} ({roi:.1f}% ROI)")

print(f"\n   Recommended Threshold: {optimal_threshold:.2f}")
print(f"     - Optimized for minimum business cost")
print(f"     - Additional savings: ${threshold_df[threshold_df['Threshold']==0.5]['Total_Cost'].values[0] - optimal_cost:,.0f}")

In [ ]:
print(f"\n2. WHY THIS MODEL?")
print(f"   {'='*80}")
print(f"   - Highest ROC-AUC score: {rec_metrics['Test_ROC_AUC']:.4f}")
print(f"   - Balanced precision and recall")
print(f"   - Strong performance across all metrics")
print(f"   - Reasonable computational cost")
print(f"   - Good generalization (low train-test gap)")

train_test_gap = rec_metrics['Train_ROC_AUC'] - rec_metrics['Test_ROC_AUC']
print(f"   - Train-Test ROC-AUC gap: {train_test_gap:.4f} (low overfitting)")

In [ ]:
print(f"\n3. DEPLOYMENT RECOMMENDATIONS")
print(f"   {'='*80}")
print(f"\n   Model Configuration:")
print(f"     - Model: {recommended_model}")
print(f"     - Decision Threshold: {optimal_threshold:.2f}")
print(f"     - Model File: ../models/saved_models/{recommended_model.lower().replace(' ', '_')}_model.pkl")

print(f"\n   Implementation Strategy:")
print(f"     1. Deploy model with optimal threshold ({optimal_threshold:.2f})")
print(f"     2. Monitor false negative rate (target: <{rec_business['False_Negatives']/len(y_test):.1%})")
print(f"     3. Alert operations team when prediction probability > {optimal_threshold:.2f}")
print(f"     4. Implement intervention workflow for predicted delays")

print(f"\n   Monitoring Requirements:")
print(f"     - Track model performance weekly")
print(f"     - Monitor data drift in key features")
print(f"     - Retrain model quarterly or when AUC drops below {rec_metrics['Test_ROC_AUC']-0.05:.2f}")
print(f"     - A/B test threshold adjustments")

print(f"\n   Business Impact:")
print(f"     - Expected annual savings: ~${(savings * 52):,.0f} (assuming weekly batches)")
print(f"     - Reduced late deliveries by catching {rec_metrics['Test_Recall']:.0%} of delays")
print(f"     - Improved customer satisfaction through proactive communication")

In [ ]:
print(f"\n4. NEXT STEPS FOR PRODUCTION")
print(f"   {'='*80}")
print(f"\n   Technical:")
print(f"     □ Create prediction API/service")
print(f"     □ Implement feature pipeline for new data")
print(f"     □ Set up model versioning and rollback")
print(f"     □ Build monitoring dashboard")
print(f"     □ Implement automated retraining pipeline")

print(f"\n   Business:")
print(f"     □ Define intervention protocols for predicted delays")
print(f"     □ Train operations team on model outputs")
print(f"     □ Set up escalation workflows")
print(f"     □ Measure actual cost savings post-deployment")
print(f"     □ Gather feedback for model improvements")

print(f"\n   Data Science:")
print(f"     □ Experiment with additional features")
print(f"     □ Try ensemble of top models")
print(f"     □ Implement SHAP for detailed explainability")
print(f"     □ Analyze model performance by segment")
print(f"     □ Build separate models for different routes/products")

print(f"\n{'='*100}")

## 7. Save Final Report

In [ ]:
# Create final report summary
final_report = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d'),
    'recommended_model': recommended_model,
    'optimal_threshold': optimal_threshold,
    'test_roc_auc': rec_metrics['Test_ROC_AUC'],
    'test_precision': rec_metrics['Test_Precision'],
    'test_recall': rec_metrics['Test_Recall'],
    'test_f1': rec_metrics['Test_F1'],
    'false_negatives': int(rec_business['False_Negatives']),
    'false_positives': int(rec_business['False_Positives']),
    'total_business_cost': float(rec_business['Total_Cost']),
    'estimated_savings': float(savings),
    'roi_percentage': float(roi)
}

import json
with open('../models/model_metadata/final_model_report.json', 'w') as f:
    json.dump(final_report, f, indent=2)

print("Final report saved to: ../models/model_metadata/final_model_report.json")
print("\nProject completed successfully!")